## Demo file on training the auto-encoder model

In [1]:
import numpy as np

In [2]:
sample = np.load("trim_sample.npy",allow_pickle=True)

In [3]:
sample.shape

(4325, 30000)

In [4]:
train = sample[:100,]
test  =  sample[2000:2100,]

In [5]:
train.shape

(100, 30000)

In [6]:
from autoencode import Auto_Encoder
import torch
import torchvision
from torch import nn, optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import os

In [28]:
learning_rate = 0.001
batch_size = 64
epochsize = 30

In [29]:
AE = Auto_Encoder()
MSELoss = nn.MSELoss()
optimizer = optim.Adam(AE.parameters(), lr=learning_rate)

In [30]:
AE.parameters()

<generator object Module.parameters at 0x4fc588d60>

In [31]:
Train = torch.from_numpy(train).to(torch.float32)
Train.shape

torch.Size([100, 30000])

In [32]:
loss1 = MSELoss (AE(Train[0]),Train[0])

In [33]:
loss1

tensor(0.1226, grad_fn=<MseLossBackward0>)

In [34]:
### Train
for s in Train:
    s_prime = AE(s)

    # loss 
    loss = MSELoss(s_prime, s)
    #print(loss)
    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [35]:
loss2 = MSELoss (AE(Train[0]),Train[0])

In [36]:
loss2

tensor(130.5496, grad_fn=<MseLossBackward0>)

In [37]:
loss1 - loss2

tensor(-130.4270, grad_fn=<SubBackward0>)

In [38]:
### testing
losses = []
Test = torch.from_numpy(test).to(torch.float32)
for s in Test:
    s_prime = AE(s)

    # loss 
    loss = MSELoss(s_prime, s)
    losses.append((loss.detach().numpy()))
    #print(loss)
    # update
    

In [39]:
np.mean(losses)

8.0948715

In [40]:
sample1 = Test[0]
sample1 = sample1.reshape(-1,3)

In [41]:
recover_sample = AE(Test[0]).reshape(-1,3)

In [42]:
import open3d as o3d

In [43]:
pcd1 = o3d.geometry.PointCloud()
pcd2 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(sample1.detach().numpy()) 
pcd2.points = o3d.utility.Vector3dVector(recover_sample.detach().numpy() )

In [44]:
o3d.visualization.draw_geometries([pcd1])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [45]:
o3d.visualization.draw_geometries([pcd2])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [46]:
np.sum(recover_sample.detach().numpy() != 0)

30000

In [47]:
recover_sample[recover_sample!=0]

tensor([ 0.6814, -2.2303, -0.7627,  ...,  0.3119, -0.2306, -2.1949],
       grad_fn=<IndexBackward0>)

: 